# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize
import mpl_toolkits.mplot3d

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import inauguralproject
from HouseholdSpecializationModel import  HouseholdSpecializationModelClass



In [ ]:
model = HouseholdSpecializationModelClass()

# set values of alpha and sigma
alpha_values = [0.25, 0.5, 0.75]
sigma_values = [0.5, 1, 1.5]

# create empty lists
HM_list = []
HF_list = []

# loop over alpha and sigma values
for alpha in alpha_values:
    for sigma in sigma_values:
        
        # set values of alpha and sigma in model parameters
        model.par.alpha = alpha
        model.par.sigma = sigma
        
        # solve model discretely
        opt = model.solve_discrete()
        
        # append optimal HM and HF values to lists
        HM_list.append(opt.HM)
        HF_list.append(opt.HF)
        
# print lists of HM and HF values
#print("HM values:", HM_list)
#print("HF values:", HF_list)


# Converting to arrays to use numpy functions
HM_array = np.array(HM_list)
HF_array = np.array(HF_list)

# Finding the ratio
HF_HM_ratio = HF_array / HM_array

# Plotting
#print("HF / HM Ratio", HF_HM_ratio)

# Reshaping the array to a matrix with alpha on the rows and sigma on the columns
HF_HM_matrix = HF_HM_ratio.reshape(len(alpha_values), len(sigma_values))

# Plotting the ratios for the different alphas with sigma as a fixed variable
for i in range(len(sigma_values)):
    plt.plot(alpha_values, HF_HM_matrix[:,i], label='Sigma='+str(sigma_values[i]))

plt.xlabel('Alpha')
plt.ylabel('HF/HM Ratio')
plt.title('HF/HM Ratio for different alpha and sigma values')
plt.legend()
plt.show()



# Question 2

plotting $Wf/Wm$ ratio

In [ ]:
# Create arrays for WF/WM ratio
WFlist = np.linspace(0.8, 1.2, 9)
WM = np.ones(9)
WF = np.array(WFlist)
WF_WM_r = WF / WM

# Take log of values
log_wf_wm_r = np.log(WF_WM_r)
log_hf_hm = np.log(HF_HM_ratio)

# Plot
print(len(log_wf_wm_r), len(log_hf_hm))
plt.scatter(log_hf_hm, log_wf_wm_r)
slope, intercept = np.polyfit(log_hf_hm, log_wf_wm_r, 1)
plt.plot(log_hf_hm, slope * log_hf_hm + intercept, color='blue')
plt.xlabel("Log of Hf/Hm Ratio")
plt.ylabel("Log of Wf/Wm Ratio")
plt.title("Log of Hf/Hm and Wf/Wm")
plt.show()

# Question 3

Plot the $Wf / Wm$ ratio for the continuous values 

In [ ]:
model = HouseholdSpecializationModelClass()

# set values of alpha and sigma
alpha_values = [0.25, 0.5, 0.75]
sigma_values = [0.5, 1, 1.5]

# create empty lists
HM_list = []
HF_list = []

# loop over alpha and sigma values
for alpha in alpha_values:
    for sigma in sigma_values:
        
        # set values of alpha and sigma in model parameters
        model.par.alpha = alpha
        model.par.sigma = sigma
        
        # solve model continuously
        opt = model.solve()
        
        # append optimal HM and HF values to lists
        HM_list.append(opt.HM)
        HF_list.append(opt.HF)
        
# print lists of HM and HF values
#print("HM values:", HM_list)
#print("HF values:", HF_list)


# Converting to arrays to use numpy functions
HM_array = np.array(HM_list)
HF_array = np.array(HF_list)

# Finding the ratio
HF_HM_ratio = HF_array / HM_array

# Plotting
#print("HF / HM Ratio", HF_HM_ratio)

# Reshaping the array to a matrix with alpha on the rows and sigma on the columns
HF_HM_matrix = HF_HM_ratio.reshape(len(alpha_values), len(sigma_values))

# Plotting the ratios for the different alphas with sigma as a fixed variable
for i in range(len(sigma_values)):
    plt.plot(alpha_values, HF_HM_matrix[:,i], label='Sigma='+str(sigma_values[i]))

plt.xlabel('Alpha')
plt.ylabel('HF/HM Ratio')
plt.title('HF/HM Ratio for different alpha and sigma values')
plt.legend()
plt.show()


# Question 4

# Question 5

ADD CONCISE CONLUSION.